Kita main pakai NLTK !!

**1. Import libraries yang kita gunakan, jangan lupa unduh BROWN, Treebank, Punkt**


In [ ]:
import nltk
nltk.download('brown')
nltk.download('treebank')
nltk.download('punkt')

from nltk import word_tokenize
from nltk.corpus import brown as cb
from nltk.corpus import treebank as tb

import pprint
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


**2. Kita coba lihat dulu datasetkita; Corpus BROWN**

In [ ]:
data = nltk.Text(cb.words('ca01')) # membaca kalimat ke ca0...
print (data)

<Text: The Fulton County Grand Jury said Friday an...>


In [ ]:
print (cb.words('ca01')[0:20])

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that']


In [ ]:
print (cb.tagged_words('ca01')[0:5]) # melihat kategori tagging words

[('The', 'AT'), ('Fulton', 'NP-TL'), ('County', 'NN-TL'), ('Grand', 'JJ-TL'), ('Jury', 'NN-TL')]


In [ ]:
tagged_sentences_brown_corpus = nltk.corpus.brown.tagged_sents()
pprint.pprint (tagged_sentences_brown_corpus[0])

[('The', 'AT'),
 ('Fulton', 'NP-TL'),
 ('County', 'NN-TL'),
 ('Grand', 'JJ-TL'),
 ('Jury', 'NN-TL'),
 ('said', 'VBD'),
 ('Friday', 'NR'),
 ('an', 'AT'),
 ('investigation', 'NN'),
 ('of', 'IN'),
 ("Atlanta's", 'NP$'),
 ('recent', 'JJ'),
 ('primary', 'NN'),
 ('election', 'NN'),
 ('produced', 'VBD'),
 ('``', '``'),
 ('no', 'AT'),
 ('evidence', 'NN'),
 ("''", "''"),
 ('that', 'CS'),
 ('any', 'DTI'),
 ('irregularities', 'NNS'),
 ('took', 'VBD'),
 ('place', 'NN'),
 ('.', '.')]


**3. Kita coba eksplor corpus Penn-Treebank**

In [ ]:
data = nltk.Text(tb.words()[0:10])
print (data)

<Text: Pierre Vinken , 61 years old , will...>


In [ ]:
print (tb.words()[0:20])

['Pierre', 'Vinken', ',', '61', 'years', 'old', ',', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'Nov.', '29', '.', 'Mr.', 'Vinken']


In [ ]:
tagged_sentences_treebank_corpus = nltk.corpus.treebank.tagged_sents()
pprint.pprint (tagged_sentences_treebank_corpus[0])

[('Pierre', 'NNP'),
 ('Vinken', 'NNP'),
 (',', ','),
 ('61', 'CD'),
 ('years', 'NNS'),
 ('old', 'JJ'),
 (',', ','),
 ('will', 'MD'),
 ('join', 'VB'),
 ('the', 'DT'),
 ('board', 'NN'),
 ('as', 'IN'),
 ('a', 'DT'),
 ('nonexecutive', 'JJ'),
 ('director', 'NN'),
 ('Nov.', 'NNP'),
 ('29', 'CD'),
 ('.', '.')]


**4. Kita pilih corpus Penn-Treebank untuk bangun / bikin POS Tagger sendiri**

In [ ]:
print ("Kalimat yang di-tag sebanyak :", len(tagged_sentences_treebank_corpus))

Kalimat yang di-tag sebanyak : 3914


In [ ]:
print ("Jumlah kata yang di-tag sebanyak :", len(nltk.corpus.treebank.tagged_words()))

Jumlah kata yang di-tag sebanyak : 100676


Kita buat fitur pakai fungsi!!

In [ ]:
def features(sentence, index) :
  return {
      'word' : sentence[index],
      'is_first' : index == 0,
      'is_last' : index == len(sentence) - 1,
      'is_capitalized' : sentence[index][0].upper() == sentence[index][0],
      'is_all_caps' : sentence[index].upper() == sentence[index],
      'is_all_lower' : sentence[index].lower() == sentence[index],

      'prefix-1' : sentence[index][0],
      # 'prefix-2' : sentence[index][:2],
      # 'prefix-3' : sentence[index][:3],

      'sufix-1' : sentence[index][-1],
      # 'sufix-2' : sentence[index][-2],
      # 'sufix-3' : sentence[index][-3],

      'prev_word' : '' if index == 0 else sentence[index - 1],
      'next_word' : '' if index == len(sentence) - 1 else sentence[index + 1],
      'has_hypen' : '-' in sentence[index],
      'is_numeric' : sentence[index].isdigit(),
      'capitals_inside' : sentence[index][1:].lower() != sentence[index][1:]
  }

In [ ]:
pprint.pprint(features(['This', 'IS', 'a', 'sentence'], 0))

{'capitals_inside': False,
 'has_hypen': False,
 'is_all_caps': False,
 'is_all_lower': False,
 'is_capitalized': True,
 'is_first': True,
 'is_last': False,
 'is_numeric': False,
 'next_word': 'IS',
 'prefix-1': 'T',
 'prev_word': '',
 'sufix-1': 's',
 'word': 'This'}


**5. Transformasi dataset!**

In [ ]:
def untag(tagged_sentences) :
  return [w for w, t in tagged_sentences]

Ubah dataset menjadi pasangan X dan y!!


X = Fitur

y = label POS

In [ ]:
def transform_to_dataset(tagged_sentences) :
  X, y = [], []
  for tagged in tagged_sentences :
    for index in range(len(tagged)) :
      X.append(features(untag(tagged), index))
      y.append(tagged[index][1])
  return X,y

**6. Bagi dataset ke training dan testing!**

In [ ]:
cutoff = int(.75 * len(tagged_sentences_treebank_corpus)) # memotong seberapa bagian data
training_sentences = tagged_sentences_treebank_corpus[:cutoff]
testing_sentences = tagged_sentences_treebank_corpus[cutoff:] # cutoff: -> mengambil sisanya

In [ ]:
print (len(training_sentences))
print (len(testing_sentences))

2935
979


In [ ]:
X, y = transform_to_dataset(training_sentences)

**7. Train Model!**

In [ ]:
clf = Pipeline([
    ('vectorizer', DictVectorizer(sparse = False)),
    ('classifier', DecisionTreeClassifier(criterion = 'entropy'))
])

In [ ]:
clf.fit (X[:10000], y[:10000])

Pipeline(steps=[('vectorizer', DictVectorizer(sparse=False)),
                ('classifier', DecisionTreeClassifier(criterion='entropy'))])

**8. Hitung Akurasi!**

In [ ]:
X_test, y_test = transform_to_dataset(testing_sentences)

In [ ]:
print ("Tingkat Akurasi : {:.2%}" . format(clf.score(X_test, y_test)))

Tingkat Akurasi : 88.55%


**9. Coba generate POS dengan kalimat suka-suka saya**

In [ ]:
def pos_tag(sentence) :
  tagged_sentences = []
  tags = clf.predict([features(sentence, index) for index in range(len(sentence))])
  return zip(sentence, tags)

In [ ]:
POS_list = list(pos_tag(word_tokenize('My NLP lecture is Evy'))) # jika di train dengan bhs inggris, testing juga harus bahasa inggris
for t in POS_list :
  print (u"{:<16}{:>2}" . format(str(t[0]), str(t[1])))

My              NNP
NLP             NNP
lecture         RB
is              VBZ
Evy             NNP
